In [1]:
import numpy as np
import scipy.sparse as sparse

import pandas as pd

import itertools

In [2]:
# Example Time Series Generator

X = np.random.randint(low=0,high=2,size=(3,1000))

X[2,:] = X[0,:] ^ X[1,:]

In [3]:
class Analysis:

    def __init__(self, data):

        # number of variables
        self.nvars = np.shape(data)[0]

        # length
        self.duration = np.shape(data)[1]

        # determine the joint alphabet
        vars = np.arange(0,self.nvars)
        alphabet = np.unique(data[vars,:],axis=1)

        # calculate the joint distribution
        self.P = pd.DataFrame(sparse.csc_matrix((np.shape(alphabet)[1],1)),index=pd.MultiIndex.from_arrays(alphabet))

        self.P.index.names = vars
        self.P.rename({0:'joint'}, axis='columns', inplace=True)
        
        for a in self.P.index:

            self.P.loc[a] = np.count_nonzero(np.all((data[vars,:]==np.reshape(np.array(a),(-1,1))),axis=0),axis=0)

        self.P['joint'] = self.P.joint.values/np.sum(self.P.joint.values)


    # calculate the marginal distribution of one variable or a higher-order marginal of several variables
    def marginal_distribution(self,x):

        return self.P.groupby(level=x).sum().rename({'joint':'marginal'}, axis='columns')

    
    # return a conditional distribution of one or several variables given one or several other variables
    def conditional_distribution(self,x,y):

        xy = tuple(list(set(x).union(set(y))))

        # Calculate the joint distribution of x and y
        p_xy = self.marginal_distribution(xy)
        p_xy.rename({'marginal':'joint'}, axis='columns', inplace=True)
        p_xy.reset_index(inplace=True)
        
        # Calculate the marginal of the conditioning variable
        p_y = self.marginal_distribution(y)
        p_y.reset_index(inplace=True)

        p_xcy = p_xy.merge(p_y, left_on=y, right_on=y)

        p_xcy['conditional'] = p_xcy['joint']/p_xcy['marginal']
        p_xcy.set_index(list(xy), inplace=True)

        return p_xcy

    # Calculate the entropy of one or more variables
    def entropy(self,var=None):

        if np.all(var==None):

            p = self.P['joint'][self.P['joint'].ne(0)].values.astype('float')

        else:

            p = self.P.groupby(level=var).sum().values
        
        return -np.sum(p*np.log2(p))


    # Conditional Entropy
    def conditional_entropy(self,x,y):

        xy = list(set(x).union(set(y)))

        H_XY = self.entropy(xy)
        H_Y  = self.entropy(y)

        return H_XY  - H_Y
        
    # Mutual Information
    def mutual_information(self,x,y):

        xy = list(set(x).union(set(y)))

        H_X = self.entropy(x)
        H_Y = self.entropy(y)

        H_XY = self.entropy(xy)

        return H_X + H_Y - H_XY
    

    # Interaction Information
    def interaction_information(self):

        II = 0

        vars = np.arange(0,self.nvars)

        for n_subset in range(1,self.nvars+1):

            for subset in [list(sub) for sub in itertools.combinations(vars, r=n_subset)]:

                II += (-1)**(self.nvars-n_subset+1)*self.entropy(subset)


        return II


    # total correlation
    def total_correlation(self):

        vars = list(range(0,self.nvars))

        HS = self.entropy(vars)

        HX = np.sum([self.entropy([x]) for x in vars])

        return HX - HS


    # dual total correlation
    def dual_total_correlation(self):

        vars = np.arange(0,self.nvars)

        HS = self.entropy(list(vars))

        HX = np.sum([self.entropy(list(vars[vars!=x])) for x in vars])

        return HX - (self.nvars-1)*HS


    # redundancy synergy index
    def redundancy_synergy_index(self,source,target):

        I_joint = self.mutual_information(source,target)

        I_ind =  np.sum([self.mutual_information([x],target) for x in source])

        return I_joint - I_ind

    
    # varadans synergy
    def varadans_synergy(self,source, target, source_partitions):

        I_joint = self.mutual_information(source,target)

        I_max = max([sum([self.mutual_information(p,target) for p in partition]) for partition in source_partitions])

        return I_joint - I_max


    # Delta I
    def delta_I(source,target):

        p_xcy = self.conditional([source[0]],target)[['conditional']]

        print(p_xcy)

        


In [4]:
source = [0,1]
target = [2]

# Get the distribution of x_i given y
p_xicy = x.conditional_distribution([source[0]],target)[['conditional']]
p_xicy.rename({'conditional':source[0]}, axis=1,inplace=True)
p_xicy.index.names = ['s',*['t_'+str(t) for t in target]]

if len(source)>1:

    for s in source[1:]:

        temp = x.conditional_distribution([s],target)[['conditional']]
        temp.rename({'conditional':s}, axis=1,inplace=True)
        temp.index.names = ['source','target']
        temp.index.names = ['s',*['t_'+str(t) for t in target]]

        p_xicy = pd.merge(p_xicy, temp, left_index=True, right_index=True, how='outer')
    
p_xicy = p_xicy.fillna(0)

# Determine the conditional distribution of x given y under the independent model
p_ind_xcy = x.conditional_distribution(source,target)[['conditional']]
p_ind_xcy.rename({'conditional':'ind_conditional'}, axis='columns', inplace=True)
p_ind_xcy.loc[:,'ind_conditional'] = 0

id = len(source)

for vars in p_ind_xcy.index:

    p_ind_xcy.loc[vars,'ind_conditional'] = np.prod([p_xicy.loc[(a,*vars[id:]),source[ia]] for ia,a in enumerate(vars[:id])])

# Estimate the independent probability of x
p_y = x.marginal_distribution(target)

p_ind_x = pd.merge(p_ind_xcy.reset_index(), p_y.reset_index(), on=target, how='left')
p_ind_x.set_index([*source,*target], inplace=True)
p_ind_x.rename({'ind_conditional':'x_conditional_y','marginal':'marginal_y'}, axis='columns', inplace=True)
p_ind_x['marginal_x'] = p_ind_x['x_conditional_y']*p_ind_x['marginal_y']
p_ind_x = p_ind_x.reset_index().groupby(source).sum().drop([*target,'x_conditional_y','marginal_y'], axis='columns')

# Estimate y conditional x under the independent model
p_ind_ycx = x.P.copy()
p_ind_ycx.rename({'joint':'y_ind_conditional_x'}, axis='columns', inplace=True)
p_ind_ycx.loc[:,'y_ind_conditional_x'] = 0

temp = pd.merge(p_ind_xcy.reset_index(), p_y.reset_index(), on=target, how='left')
temp.rename({'marginal':'marginal_y','ind_conditional':'x_ind_conditional_y'},axis='columns',inplace=True)

p_ind_ycx = pd.merge(p_ind_ycx.reset_index(), temp, on=[*source,*target], how='left')
p_ind_ycx = pd.merge(p_ind_ycx, p_ind_x.reset_index(), on=source, how='left')

p_ind_ycx.set_index([*source, *target], inplace=True)

p_ind_ycx.loc[:,'y_ind_conditional_x'] = p_ind_ycx['x_ind_conditional_y']*p_ind_ycx['marginal_y']/p_ind_ycx['marginal_x']
p_ind_ycx = p_ind_ycx.fillna(0)

# estimate the true conditional distribution
p_ycx = x.conditional_distribution(target,source)[['conditional']]

p_ycx.mer

NameError: name 'x' is not defined

In [68]:
y = [2]
x = [0,1]

# Calculate the empirical joint, conditional and y-marginal
p = self1.conditional_distribution(x,y)
p.rename({'joint':'xy','marginal':'y','conditional':'x|y'}, axis='columns', inplace=True)

# Calculate the distribution of each xi conditioned on y
for xi in x:

    p_xicy = self1.conditional_distribution([xi],y)[['conditional']]
    p_xicy.rename({'conditional':'x'+str(xi)+'|y'}, axis='columns', inplace=True)

    p = pd.merge(p.reset_index(), p_xicy.reset_index(), left_on=[xi,*y], right_on=[xi,*y], how='left')
    p.set_index([*x,*y], inplace=True)

# Calculate the conditional distribution of x given y under the independent model
p['ind_x|y'] = np.ones(p.shape[0])

for xi in x:

    p.loc[:,'ind_x|y'] *= p.loc[:,'x'+str(xi)+'|y']

# Note that this skips those contributions to the
p


,,,xy,y,x|y,x0|y,x1|y,ind_x|y
0,1,2,,,,,,
0,0,0,0.243,0.476,0.510504,0.510504,0.510504,0.260615
1,1,0,0.233,0.476,0.489496,0.489496,0.489496,0.239606
0,1,1,0.261,0.524,0.498092,0.498092,0.498092,0.248095
1,0,1,0.263,0.524,0.501908,0.501908,0.501908,0.251912


In [53]:
list(itertools.product([0,1],[0,1],[0,1]))

[(0, 0, 0),
 (0, 0, 1),
 (0, 1, 0),
 (0, 1, 1),
 (1, 0, 0),
 (1, 0, 1),
 (1, 1, 0),
 (1, 1, 1)]

In [67]:
p.loc[idx[0,:,1],'x0|y'].values[0]

0.49809160305343514

In [64]:
from pandas import IndexSlice as idx

In [128]:
p_ind_xcy

,,,ind_conditional
0,1,2,
0,0,0,0.240770
1,1,0,0.259404
0,1,1,0.233829
1,0,1,0.266711


In [132]:
p_ind_ycx

joint  y_ind_conditional_x
0 1 2                            
0 0 0  0.237                    0
  1 1   0.25                    0
1 0 1  0.267                    0
  1 0  0.246                    0

0  1  2
0  0  0    1.0
1  1  0    1.0
0  1  1    1.0
1  0  1    1.0
dtype: float64

In [98]:
p_ind_xcy

,,,independent
0,1,2,
0,0,0,0.240770
1,1,0,0.259404
0,1,1,0.233829
1,0,1,0.266711


In [76]:
a = [1,2]
a[:2]

[1, 2]

In [ ]:
p.loc[0,0]*p_loc[0,1]

In [72]:
p_xicy

,,0,1
s,t_2,,
0,0,0.490683,0.490683
1,0,0.509317,0.509317
0,1,0.483559,0.516441
1,1,0.516441,0.483559


In [71]:
p_ind_xcy

,,,independent
0,1,2,
0,0,0,0
1,1,0,0
0,1,1,0
1,0,1,0


In [46]:
x = Analysis(X)
x.varadans_synergy([0,1],[2],source_partitions=partitions)

0.9986512539479018

In [913]:
class TimeSeries:

    def __init__(self, data):

        # store data
        self.data = data

        # number of variables
        self.nvars = np.shape(data)[0]

        # length
        self.duration = np.shape(data)[1]

        
    # distribution estimation
    def estimate_distribution(self,vars):

        # determine the (joint)-alphabet
        alphabet = np.unique(self.data[vars,:],axis=1)
        
        # count occurance of each (joint)-symbol
        counts = np.zeros(np.shape(alphabet)[1])

        for ia in range(0,np.shape(alphabet)[1]):

            counts[ia] = np.count_nonzero(np.all((self.data[vars,:]==alphabet[:,[ia]]),axis=0),axis=0)
        
        # normalize
        distribution = counts/np.sum(counts)

        return distribution, alphabet

    
    # entropy
    def entropy(self, vars):

        vars = np.array(vars)

        H = np.zeros(len(vars))

        for ix,x in enumerate(vars):

            # estimate the distribution of the variable
            p, _ = self.estimate_distribution([x])

            # avoid 0 in log
            p[p==0] = 1

            # calculate the entropy
            H[ix] = -np.sum(p*np.log2(p))

        return H

    
    # joint entropy
    def joint_entropy(self,vars):

        vars = np.array(vars)

        # estimate the distribution of the variable
        p, _ = self.estimate_distribution(vars)

        # avoid 0 in log
        p[p==0] = 1

        # calculate the entropy
        return -np.sum(p*np.log2(p))


    # conditional entropy
    def conditional_entropy(self,x,y):

        # take care of variables that occur on both sides
        xy = np.array(list(set(x).union(set(y))))

        # determine the joint entropy
        Hxy = self.joint_entropy(xy)

        # determine the entropy of the conditioning variable
        Hx =  self.joint_entropy(x)


        return Hxy - Hx

    
    # mutual information
    def mutual_information(self,x,y):

        # take care of variables that appear on both sides
        xy = np.array(list(set(x).union(set(y))))

        # estimate the entropy of the x variables
        Hx = self.joint_entropy(x)

        # estimate the entropy of the y variables
        Hy = self.joint_entropy(y)

        # estimate the joint entropy of x and y variables
        Hxy = self.joint_entropy(xy)

        # calculate the mutual information
        return Hx + Hy - Hxy


    # interaction information
    def interaction_information(self):

        II = 0

        vars = np.arange(0,self.nvars)

        for n_subset in range(1,self.nvars+1):

            for subset in [np.array(sub) for sub in itertools.combinations(vars, r=n_subset)]:

                II += (-1)**(self.nvars-n_subset+1)*self.joint_entropy(subset)

        return II


    # total correlation
    def total_correlation(self):

        vars = np.arange(0,self.nvars)

        HS = self.joint_entropy(vars)

        HX = np.sum([self.entropy([x]) for x in vars])

        return HX - HS


    # dual total correlation
    def dual_total_correlation(self):

        vars = np.arange(0,self.nvars)

        HS = self.joint_entropy(vars)

        HX = np.sum([self.joint_entropy(vars[vars!=x]) for x in vars])

        return HX - (self.nvars-1)*HS

    # redundancy synergy index
    def redundancy_synergy_index(self,source,target):

        source = np.array(source)

        I_joint = self.mutual_information(source,target)

        I_ind =  np.sum([self.mutual_information([x],target) for x in source])

        return I_joint - I_ind

    # varadans synergy
    def varadans_synergy(self,source, target, source_partitions):

        source = np.array(source)

        I_joint = self.mutual_information(source,target)

        I_max = max([sum([self.mutual_information(p,target) for p in partition]) for partition in source_partitions])

        return I_joint - I_max

    # Delta I
    def delta_I(source,target):

        p_ind 

        for x in source:

            xx

In [800]:
vars = [0,1]
target = [2]

# Estimate the target variable distribution
p_y, alph_y = P.estimate_distribution(target)

# Estimate the predictor variable distribution
p_x, alph_x = P.estimate_distribution(vars)

# 
joint = np.array(list(set(vars).union(set(target))))

# Estimate the joint-distribution of target and estimators
p_xy, alph_xy = P.estimate_distribution(joint)

p_xcy = np.zeros(np.array(np.shape(alph_xy))-np.array([1,0]))
alph_xcy = alph_xy

for x in [0,1]:

    p_xiy,alph_xiy = P.estimate_distribution([x,*target])

    for i in range(0,np.shape(alph_xy)[1]):

        # find the spot where to write to the conditional
        id_xiy = np.all(alph_xiy==alph_xcy[[x,*target]][:,[i]],axis=0)

        # find the normalizing probability
        id_y = np.ravel(alph_y)==alph_xcy[-1,[i]]

        # Calculate the conditional distribution of x given y
        p_xcy[x,i] = p_xiy[id_xiy]/p_y[id_y]

# calculate the probability of x given y in the independent model
p_ind_xcy = np.prod(p_xcy,axis=0)

# Invert with Bayes Theorem


In [805]:
p_ind_x = np.zeros_like(p_x)

for ix in range(0,np.shape(alph_x)[1]):

    for iy in range(0,np.shape(alph_y)[1]):

        id_ind_xcy = np.all(alph_xcy== np.vstack([alph_x[:,[ix]],alph_y[0][iy]]),axis=0)

        if np.any(id_ind_xcy):
            p_ind_x[ix] += p_ind_xcy[id_ind_xcy]*p_y[iy]

array([[0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.]])

In [41]:
partitions = [[[0],[1,2]],
              [[1],[0,2]],
              [[2],[0,1]],
              [[0],[1],[2]]]

partitions = [[[0],[1]]]